In [50]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ydata_profiling as yp
from sklearn.model_selection import train_test_split

In [51]:
data = pd.read_csv('./data/covtype.data', sep=',', header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


In [52]:
y = data.iloc[:, -1]

wilderness_areas = data.iloc[:, 10:14]
wilderness_areas.columns = list(range(1, 5))
data['10'] = wilderness_areas.idxmax(axis=1)

soil_types = data.iloc[:, 14:54]
soil_types.columns = list(range(1, 41))
data['11'] = soil_types.idxmax(axis=1)


data = data.drop(columns=data.columns[10:55])

data.head()


,0,1,2,3,4,5,6,7,8,9,10,11
0,2596,51,3,258,0,510,221,232,148,6279,1,29
1,2590,56,2,212,-6,390,220,235,151,6225,1,29
2,2804,139,9,268,65,3180,234,238,135,6121,1,12
3,2785,155,18,242,118,3090,238,238,122,6211,1,30
4,2595,45,2,153,-1,391,220,234,150,6172,1,29


In [55]:
X = data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(464809, 12)
(116203, 12)
(464809,)
(116203,)


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler

pipe = Pipeline([('scaler', StandardScaler()), ('clf', GradientBoostingClassifier())])

pipe.fit(X_train, y_train)

accuracy = pipe.score(X_test, y_test)
print(accuracy)